In [56]:
import os
os.environ['DEBUG'] = '1'

import numpy as np

from tinygrad.tensor import Tensor
from tinygrad.nn import Linear

from lib.utils import get_mnist
from tinygrad.nn.optim import SGD

In [57]:
X_train, Y_train, X_test, Y_test = get_mnist("../data") # these need to be tensors??

In [58]:
class TinyNet:
    def __init__(self):
        self.l1 = Linear(784, 128, bias=False)
        self.l2 = Linear(128, 10, bias=False)

    def __call__(self, x):
        x = self.l1(x)
        x = x.leakyrelu()
        x = self.l2(x)
        return x

In [63]:
model = TinyNet()
opt = SGD([model.l1.weight, model.l2.weight], lr=3e-4)

In [64]:
with Tensor.train():
    for step in range(1000):
        # random sample a batch
        samp = np.random.randint(0, X_train.shape[0], size=(64))
        
        # get batch and labels
        batch = Tensor(X_train[samp], requires_grad=False)
        labels = Tensor(Y_train[samp])

        out = model(batch) # forward pass
        loss = out.sparse_categorical_crossentropy(labels) # calculate loss
        opt.zero_grad() # zero out gradients
        loss.backward() # backward pass
        opt.step() # update weights

        # calculate accuracy
        pred = out.argmax(axis=-1)
        acc = (pred == labels).mean()

        if step % 100 == 0:
            print(f"Step {step+1} | Loss: {loss.numpy():0.4f} | Accuracy: {acc.numpy():0.4f}")

Step 1 | Loss: 2.2972 | Accuracy: 0.0938
Step 101 | Loss: 2.2910 | Accuracy: 0.1719
Step 201 | Loss: 2.2904 | Accuracy: 0.1562
Step 301 | Loss: 2.2843 | Accuracy: 0.1875
Step 401 | Loss: 2.2783 | Accuracy: 0.1250
Step 501 | Loss: 2.2607 | Accuracy: 0.2500
Step 601 | Loss: 2.2679 | Accuracy: 0.1406
Step 701 | Loss: 2.2455 | Accuracy: 0.2500
Step 801 | Loss: 2.2818 | Accuracy: 0.1250
Step 901 | Loss: 2.2522 | Accuracy: 0.3594


In [65]:
model = TinyNet()
opt = SGD([model.l1.weight, model.l2.weight], lr=3e-4)

In [71]:
class SimpleDataLoader:
    def __init__(self, X, Y, batch_size=64, shuffle=True):
        self.X = X
        self.Y = Y
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __iter__(self):
        if self.shuffle:
            indices = np.random.permutation(len(self.X)) # random indices
        else:
            indices = np.arange(len(self.X)) # ordered indices

        for start_idx in range(0, len(self.X), self.batch_size):
            end_idx = min(start_idx + self.batch_size, len(self.X))
            batch_indices = indices[start_idx:end_idx]

            yield Tensor(self.X[batch_indices]), Tensor(self.Y[batch_indices])

    def __len__(self):
        return len(self.X) // self.batch_size

In [72]:
trainloader = SimpleDataLoader(X_train, Y_train)
testloader = SimpleDataLoader(X_test, Y_test, shuffle=False)

In [74]:
for epoch in range(1000):
    with Tensor.train():
        running_train_loss = 0.0
        for x_batch, y_batch in trainloader:
            out = model(x_batch) # forward pass
            loss = out.sparse_categorical_crossentropy(y_batch) # calculate loss
            opt.zero_grad() # zero out gradients
            loss.backward() # backward pass
            opt.step() # update weights

            running_train_loss += loss.numpy()

    running_test_loss = 0.0
    for x_batch, y_batch in testloader:
        out = model(x_batch)
        loss = out.sparse_categorical_crossentropy(y_batch)
        running_test_loss += loss.numpy()

    print(f"Epoch {epoch+1} | Train Loss: {running_train_loss/len(trainloader):0.4f} | Test Loss: {running_test_loss/len(testloader):0.4f}")

Epoch 1 | Train Loss: 1.2211 | Test Loss: 1.1682
Epoch 2 | Train Loss: 1.1640 | Test Loss: 1.1115
Epoch 3 | Train Loss: 1.1116 | Test Loss: 1.0596
Epoch 4 | Train Loss: 1.0636 | Test Loss: 1.0121
Epoch 5 | Train Loss: 1.0197 | Test Loss: 0.9689
Epoch 6 | Train Loss: 0.9798 | Test Loss: 0.9295
Epoch 7 | Train Loss: 0.9433 | Test Loss: 0.8936
Epoch 8 | Train Loss: 0.9103 | Test Loss: 0.8608
Epoch 9 | Train Loss: 0.8799 | Test Loss: 0.8308
Epoch 10 | Train Loss: 0.8520 | Test Loss: 0.8034
Epoch 11 | Train Loss: 0.8265 | Test Loss: 0.7783
Epoch 12 | Train Loss: 0.8031 | Test Loss: 0.7550
Epoch 13 | Train Loss: 0.7814 | Test Loss: 0.7337
Epoch 14 | Train Loss: 0.7614 | Test Loss: 0.7139
Epoch 15 | Train Loss: 0.7427 | Test Loss: 0.6956
Epoch 16 | Train Loss: 0.7257 | Test Loss: 0.6787
Epoch 17 | Train Loss: 0.7095 | Test Loss: 0.6629


KeyboardInterrupt: 